# Basics
This example works with RDD. The purpose of this is example is solely get to know the technology. You will probably not need RDD's in your project as we wil use Spark SQL which is more flexible to use.

In [1]:
import datetime
#Make sure pyspark is installed as a package of your project.
from pyspark import SparkConf, SparkContext
import ConnectionConfig as cc
# This method will setup the environment variables for you. See EnvironmentSetup.py for more information.
cc.setupEnvironment()


# Basic setup to be able to use a Spark Cluster

In [2]:

#1. Creating a configuration to add parameters.
conf = SparkConf().setAppName("firstJob").setMaster("local[*]").setIfMissing("spark.logLineage", "true")
#setMaster() is used to define on which cluster the code has to run.
#   If you submit a job on the cluster itself the Master will be already set
#   If you want to run the code in your development environment, you have to set the master to local[*]. Spark will initiate a local-cluster that uses the different threads of your CPU.
# IMPORTANT: Never use setMaster("local[*]") hard-coded in your final code. When running on a real environment, overwriting the default with local[*] means the job will just run locally on the master node and the cluster itself will not be used to run the job.

#2. Create a sparkcontext. The context is used to initiate a processing job on the cluster
sc =SparkContext.getOrCreate((conf))
sc.uiWebUrl #just returns the sparkUI string. Visit the link below to get insights in the spark jobs

'http://host.docker.internal:4040'

## Running WordCount (the Hello World of distributed processing)

In [9]:
import datetime

# 1. Use the Spark Context to read from an input. This is the start point for the Spark excecution engine. Reading from a textFile returns a dataframe with a record for each line.
# TODO: Use the correct path notation (eg. file:///, gs://, hdfs://) and add this to the arguments of the job instead of hard-coded.
lines = sc.textFile("file:///C:/DevProjects/Databricks/FileStore/tables/shakespeare.txt")\
#2. The first transformation transforms the lines to words
words = lines.flatMap(lambda line: line.split(" "))
#3. The second transformations creates a keyvalue map: map to key= word, value= 1
wordKv = words.map(lambda word: (word, 1))
#4. In this transformations an aggregation by key is performed. Every word is counted. Because records with the same key ar spread over the cluster, a shuffle is needed (e.g. all records with key "romeo" will be sent to one node). The execution engine will start a new stage for this step because of the shuffle.

wordCounts = wordKv.reduceByKey(lambda a,b:a +b)
#5. This step is not necessary but will cache the result to memory.
# This is more efficient if the wordCounts dataframe is needed for more new dataframes to avoid that Spark recalculates everything from the beginning for each chain.
wordCounts.cache()
# 6. Save the counts to output. As this is an action, the Spark execution engine will only now decide that all the preceding transformations do need to be executed.
#print(wordCounts.saveAsTextFile("file:///C:/DevProjects/Databricks/output/words" + datetime.datetime.now().strftime("%m%d%Y%H%M%S")))
print(wordCounts.saveAsTextFile("./DevProjects/Databricks/output/words" ))

PythonRDD[62] at RDD at PythonRDD.scala:53

Py4JJavaError: An error occurred while calling o301.saveAsTextFile.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:106)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1091)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:410)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1089)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopFile$4(PairRDDFunctions.scala:1062)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScop

In [4]:

# Stops the spark context so you can build a new one.
sc.stop()